<a href="https://colab.research.google.com/github/hannaofficial/FakeNewsDetection/blob/main/FakeNewsDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

In [ ]:
import torch

import pandas as pd

from sklearn.model_selection import train_test_split


from transformers import Trainer, TrainingArguments, GPT2Tokenizer, GPT2ForSequenceClassification
from sklearn.metrics import accuracy_score

In [ ]:
fake_df_1 = pd.read_csv('Fake.csv')
true_df_1 = pd.read_csv('True.csv')


fake_df_1['label'] = 1
true_df_1['label'] = 0

In [ ]:
fake_df_1

,title,text,subject,date,label
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",1
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",1
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",1
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",1
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",1
...,...,...,...,...,...
23476,McPain: John McCain Furious That Iran Treated ...,21st Century Wire says As 21WIRE reported earl...,Middle-east,"January 16, 2016",1
23477,JUSTICE? Yahoo Settles E-mail Privacy Class-ac...,21st Century Wire says It s a familiar theme. ...,Middle-east,"January 16, 2016",1
23478,Sunnistan: US and Allied ‘Safe Zone’ Plan to T...,Patrick Henningsen 21st Century WireRemember ...,Middle-east,"January 15, 2016",1
23479,How to Blow $700 Million: Al Jazeera America F...,21st Century Wire says Al Jazeera America will...,Middle-east,"January 14, 2016",1


from matplotlib import pyplot as plt
import seaborn as sns
fake_df_1.groupby('subject').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  counted = (series['date']
                .value_counts()
              .reset_index(name='counts')
              .rename({'index': 'date'}, axis=1)
              .sort_values('date', ascending=True))
  xs = counted['date']
  ys = counted['counts']
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = fake_df_1.sort_values('date', ascending=True)
for i, (series_name, series) in enumerate(df_sorted.groupby('subject')):
  _plot_series(series, series_name, i)
  fig.legend(title='subject', bbox_to_anchor=(1, 1), loc='upper left')
sns.despine(fig=fig, ax=ax)
plt.xlabel('date')
_ = plt.ylabel('count()')

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  counted = (series['label']
                .value_counts()
              .reset_index(name='counts')
              .rename({'index': 'label'}, axis=1)
              .sort_values('label', ascending=True))
  xs = counted['label']
  ys = counted['counts']
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = fake_df_1.sort_values('label', ascending=True)
for i, (series_name, series) in enumerate(df_sorted.groupby('subject')):
  _plot_series(series, series_name, i)
  fig.legend(title='subject', bbox_to_anchor=(1, 1), loc='upper left')
sns.despine(fig=fig, ax=ax)
plt.xlabel('label')
_ = plt.ylabel('count()')

In [ ]:
num_rows_fake = int(0.05 * len(fake_df_1))
num_rows_true = int(0.05 * len(true_df_1))
fake_df = fake_df_1.head(num_rows_fake)
true_df = true_df_1.head(num_rows_true)

In [ ]:
df = pd.concat([fake_df, true_df], ignore_index=True)

In [ ]:
df.columns

Index(['title', 'text', 'subject', 'date', 'label'], dtype='object')

In [ ]:
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

**Here I use distilgpt2 instead of gpt2 because it was causing ram error**

*model.resize_token_embeddings(len(tokenizer))   so that the len of tokenizer from the data I provide doesn't exceed model tokenizer length limit*

In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2ForSequenceClassification.from_pretrained('distilgpt2', num_labels=2)

tokenizer.add_special_tokens({'pad_token': '[PAD]'})

model.resize_token_embeddings(len(tokenizer))

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at distilgpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Embedding(50258, 768)

In [ ]:
train_encodings = tokenizer(train_df['text'].tolist(), truncation=True, padding=True)
test_encodings = tokenizer(test_df['text'].tolist(), truncation=True, padding=True)

In [ ]:
class FakeNewsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = FakeNewsDataset(train_encodings, train_df['label'].tolist())
test_dataset = FakeNewsDataset(test_encodings, test_df['label'].tolist())

Setting parameters for trainer purpose  Here I use batch_size as 1 as it was showing error as well showing ram limit reached and failed that's why I use batch size=1

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=1,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
)

def compute_metrics(pred):
  labels = pred.label_ids
  preds = pred.predictions.argmax(-1)
  acc = accuracy_score(labels, preds)
  return {
      'accuracy': acc,
  }

Training my model

In [ ]:


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()

eval_results = trainer.evaluate()
print(f"Evaluation results: {eval_results}")




Step,Training Loss
10,0.680900
20,0.707700
30,0.622500
40,0.703200
50,0.714300
60,0.638800
70,0.621600
80,0.602800
90,0.525700
100,0.729200


Evaluation results: {'eval_loss': 1.7310505029399792e-07, 'eval_accuracy': 1.0, 'eval_runtime': 16.2801, 'eval_samples_per_second': 27.58, 'eval_steps_per_second': 27.58, 'epoch': 1.0}


**Here this a function where I will use my train model to predict whether the news is fakeor not**

In [ ]:
def predict_fake_news(text):
    new_encoding = tokenizer(text, truncation=True, padding=True, return_tensors='pt')
    # Move the input data to the same device as the model
    new_encoding = {k: v.to(model.device) for k, v in new_encoding.items()}
    output = model(**new_encoding)
    predicted_label = output.logits.argmax().item()

    if predicted_label == 1:
        return "Fake news"
    else:
        return "True news"
new_text = input("Enter the news text you want to classify: ")

prediction = predict_fake_news(new_text)

print(f"The model predicts this news is: {prediction}")

Enter the news text you want to classify: Alabama is a notoriously deep red state. It s a place where Democrats always think that we have zero chances of winning especially in statewide federal elections. However, that is just what happened on Tuesday night in the Special Election to replace Senator Jeff Sessions. Doug Jones, the Democratic Senate candidate who is known in the state for prosecuting the Ku Klux Klan members who bombed a church during the Civil Rights Movement and killed four little African American girls, will be the next Senator from Alabama.
The model predicts this news is: Fake news


**Downloading my trained model so that I can use it in my website that i am developing**

In [ ]:
model.save_pretrained('./my_fake_news_model')
tokenizer.save_pretrained('./my_fake_news_model')

('./my_fake_news_model/tokenizer_config.json',
 './my_fake_news_model/special_tokens_map.json',
 './my_fake_news_model/vocab.json',
 './my_fake_news_model/merges.txt',
 './my_fake_news_model/added_tokens.json')

In [ ]:
!zip -r /content/my_fake_news_model.zip /content/my_fake_news_model

  adding: content/my_fake_news_model/ (stored 0%)
  adding: content/my_fake_news_model/vocab.json (deflated 68%)
  adding: content/my_fake_news_model/added_tokens.json (stored 0%)
  adding: content/my_fake_news_model/tokenizer_config.json (deflated 63%)
  adding: content/my_fake_news_model/model.safetensors (deflated 7%)
  adding: content/my_fake_news_model/special_tokens_map.json (deflated 74%)
  adding: content/my_fake_news_model/merges.txt (deflated 53%)
  adding: content/my_fake_news_model/config.json (deflated 51%)


In [ ]:
from google.colab import files
files.download('/content/my_fake_news_model.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>